In [1]:
%reload_ext autoreload
%autoreload 2
# %matplotlib qt

from time import time
from tqdm import tqdm
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl

from cellector.io import create_from_suite2p, save_selection
from cellector.roi_processor import RoiProcessor
from cellector.gui import SelectionGUI

In [21]:
# Next step: 
# 1. Convert the roi_processor to just accept a list of stats and have a separate index for which plane they go with
#    - also require the references to be a numpy array
# 2. Visualization methods for optimizing the hyperparameters for preprocessing
# 3. Implement an optimization method for phase_corr_eps? 
#    - Might just be as simple as the user inputing the maximum size ROI, and then measuring
#      the average power for higher frequencies...

In [8]:
s2p_dir = Path(r"C:\Users\Andrew\Documents\test_cellector\suite2p")
roi_processor = create_from_suite2p(s2p_dir, use_redcell=True, clear_existing=True)

In [20]:
gui = SelectionGUI(roi_processor)